In [2]:
from scipy.integrate import solve_ivp, odeint
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#reactor constants
L = 0.0609 #m
R_inner = 2.39/1000 #m
R_outer = 3.5/1000 #m

h2Removal = 0 #change to 1 if there is removal of hydrogen

#gas flow constants
T = 622+273 #K
P = 1 #bar or atm

sccm0 = np.array([16.16, 48.48, 0, 0, 10, 5])
mols0 = sccm0/22400 * 60
y0 = mols0/sum(mols0)
print(y0)
Ctot = P/(8.3144598 * 10**-5 * T)

u0 = sum(sccm0) * 273 / T * P / 1 / 60 #cm^3/sec
A = np.pi * ((R_outer*100)**2-(R_inner*100)**2) #cm^2 MIGHT HAVE TO ADJUST THE AREA CALC A BIT
v0 = u0 / A / 100 #m/sec

#rate equation constants
Rgas = 8.314 #J / mol·K
Easmr = 165.740 #kJ/mol
Asmr = 1.68*10**8
Eawgs = 89.23 #kJ/mol
Awgs = 9.90*10**3

Ku = 1

I = 9 #curent in Amps
F = 96485 #faradays constant
Keqsmr = (101325/100000)**2 * np.exp(-26830/T + 30.114)
Keqwgs = np.exp(4400/T - 4.036)

[0.20291311 0.60873933 0.         0.         0.12556504 0.06278252]


In [7]:
#solving the ODE system first

x_num = 10

x_pts = np.linspace(0,L,x_num)

def odes(x, y): #dy/dt = f(x)
    #NEED TO ADJUST THIS. THESE ARE NOT CONCENTRATIONS, THEY ARE MOL FRACTIONS
    Cch4 = y[0]
    Ch2o, Cco, Cco2, Ch2, Car = y[1], y[2], y[3], y[4], y[5]
    #JUST ADD TEMP AS A VARIABLE AND YOU ADD THE EQUATION AND CONSTANT IN Y
    
    v = v0*(3*y0[1] - 2*Cch4)/y0[1] # v0*(1+2X)

    rsmr = 2/R_outer*Ku*Asmr*np.exp(-Easmr*1000/Rgas/T)*(Cch4*Ch2o-(P**2)*Cco*(Ch2**3)/Keqsmr)/Ctot
    rwgs = 2/R_outer*Ku*Asmr*np.exp(-Eawgs*1000/Rgas/T)*(Ch2o*Cco2-Cco*Ch2/Keqwgs)/Ctot
    rh2r = I/(2*F*L)/Ctot #have not confirmed if this is correct

    #for the jacobian, the i is the component, and then j (ignored here) is the axis (so like are we moving in x-direction or time)
    ch4 = (-rsmr)/v
    h2o = (-rsmr - rwgs)/v
    co = (rsmr - rwgs)/v
    co2 = (rwgs)/v
    h2 = (3*rsmr + rwgs + h2Removal*rh2r)/v
    ar = 0
    return [ch4, h2o, co, co2, h2, ar]

soln = odeint(odes, y0, x_pts)

TypeError: 'float' object is not subscriptable